# 5.4 HuggingFace 오픈소스를 활용한 RAG Pipeline 구성

<div style="text-align: right"> Initial issue : 2022.10.11 </div>
<div style="text-align: right"> last update : 2022.10.11 </div>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,# 성능이 안좋은 모델을 쓸경우 청크 사이즈를 줄이자(?)
    chunk_overlap=100
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter)

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large-instruct')

/Users/sguys99/anaconda3/envs/rag/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


- https://huggingface.co/intfloat/multilingual-e5-large-instruct

In [4]:
from langchain_chroma import Chroma

collection_name = 'chroma_tax'

database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    collection_name=collection_name, # 일종의 테이블 이름
    persist_directory='./chroma_huggingface'
)

### 3. HuggingFace에서 LLM 다운로드

- 작업 시점 기준 가장 괜찮은 성능을 내는 'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct' 활용

In [5]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig( # 양자화 진행
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [6]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

chat_model = HuggingFacePipeline.from_model_id(
    model_id='LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct',
    task='text-generation',
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        do_sample=False,
        repetition_penalty=1.03
    ),
    model_kwargs={'quantization_config': quantization_config}
)

ValueError: Could not load the text-generation model due to missing dependencies.

- https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct
- 링크에 접속해서 사용동의를 하고 진행해야한다.